In [1]:
import os
import sys
import git
import pathlib

import random
import time


import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist


PROJ_ROOT_PATH = pathlib.Path(git.Repo('.', search_parent_directories=True).working_tree_dir)
PROJ_ROOT =  str(PROJ_ROOT_PATH)
if PROJ_ROOT not in sys.path:
    sys.path.append(PROJ_ROOT)

from libs.constants import model_seeds, error_seeds
from libs import utils
from libs.fitnessfns import ff_fashion_cnn2_ERR, eval_fashion_cnn2_ERR
from libs.ga.ga_utils import extract_best_worst_chromosomes

In [2]:
# set seed
this_seed = 84376
tf.random.set_seed(this_seed)
np.random.seed(this_seed)
random.seed(this_seed)
os.environ['PYTHONHASHSEED']=str(this_seed)

In [3]:
# Tensorflow Limit GPU usage
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
        

In [4]:
# Prepare dataset
# Use only test images
fashion_mnist = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()
test_images = test_images.astype('float32') / 255.0
test_images = np.expand_dims(test_images, axis=-1)

In [5]:
# Load test set
# Testing with only im_num images
im_num = 128*40#128*40=5120 #[128*78 = 9984]
test_set = (test_images[im_num:], 
            test_labels[im_num:])

In [6]:
model_seeds = model_seeds[0:3]
error_seeds = [error_seeds[0],error_seeds[2]]

In [7]:
error_profile_tag_list = []
ERR_PARAM_list = [1,-1]
MAX_FAULT_PROB_LIST = [500E-3]
error_layer_list = ["c0","c1"]

for MAX_FAULT_PROB in MAX_FAULT_PROB_LIST:
    for seed in error_seeds:
        err_tag = "LIM_" + f"{int(MAX_FAULT_PROB*1E3):02d}" + "-" +str(seed)
        error_profile_tag_list.append(err_tag)

In [8]:
def eval_error_injected_with_shuffle(model, 
                                      layer_error_profiles, 
                                      ERR_PARAM,
                                      shuffle_orders,
                                      test_set):
    # intialize to None
    error_profile_c0=None
    error_profile_c1=None
    error_profile_h0=None
    error_profile_op=None

    error_profile_c0 = layer_error_profiles["c0"]
    error_profile_c1 = layer_error_profiles["c1"]
    error_profile_h0 = layer_error_profiles["h0"]
    error_profile_op = layer_error_profiles["op"]

    
    # intialize to None
    shuffle_order_c0 = None
    shuffle_order_c1 = None
    shuffle_order_h0 = None
    shuffle_order_op = None
                    
    shuffle_order_c0 = shuffle_orders["c0"]
    shuffle_order_c1 = shuffle_orders["c1"]
    shuffle_order_h0 = shuffle_orders["h0"]
    shuffle_order_op = shuffle_orders["op"]
        
    
    avg, std = eval_fashion_cnn2_ERR(model,
                                    error_profile_c0,
                                    error_profile_c1,
                                    error_profile_h0,
                                    error_profile_op,
                                    ERR_PARAM,
                                    shuffle_order_c0,
                                    shuffle_order_c1,
                                    shuffle_order_h0,
                                    shuffle_order_op,
                                    test_set)
        
    return avg

In [9]:
def reset_dict_values():
    dummy_dict =  {"c0":None,
             "c1":None,
             "h0":None,
             "op":None}
    return dummy_dict

In [10]:
dataset_seg_type = "all"
print("DATASET TYPE:\t", dataset_seg_type)

DATASET TYPE:	 all


In [11]:
model_type = "fashion-cnn2_1024"
results = {}
results_filename = 'evaluate_ga_genes-c0_c1--LIM_500-ERR_1_-1.npy'

# one layer at a time
for model_seed in model_seeds:#model_seeds[0:2]:
    results[model_seed] = {}
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    # Original Accuracy
    error_profiles = reset_dict_values()
    ERR_PARAM = None
    shuffle_orders = reset_dict_values()
    test_set = test_set
    original_accuracy = ff_fashion_cnn2_ERR(model,
                                            error_profile_c0=None,
                                            error_profile_c1=None,
                                            error_profile_h0=None,
                                            error_profile_op=None,
                                            ERR_PARAM=None,
                                            shuffle_order_c0=None,
                                            shuffle_order_c1=None,
                                            shuffle_order_h0=None,
                                            shuffle_order_op=None,
                                            test_set=test_set,
                                            batchsize=128).numpy()
    print("Original ACC: ", original_accuracy)
    results[model_seed]["original_acc"] = original_accuracy
    
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        results[model_seed][error_profile_tag] = {}
        # Load error profile
        error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
        error_profile_filename = error_profile_tag + ".npy"
        error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
        error_profile = np.load(error_profile_file)

        for ERR_PARAM in ERR_PARAM_list:
            print("ERR_PARAM:\t", ERR_PARAM)
            results[model_seed][error_profile_tag][ERR_PARAM]={}
            for error_layer in error_layer_list:
                print("Layer: ", error_layer)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]={}
                # Reset error profiles and shuffle orders for all layers
                error_profiles = reset_dict_values()
                shuffle_orders = reset_dict_values()
                
                # Inject error and observe degradation
                error_profiles[error_layer] = error_profile
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["faulty"] = eval_acc
                np.save(results_filename, results) 
                print("Faulty ACC: ", eval_acc)
                
                # Use the best chromosome and observe recovered value
                best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                   error_layer, 
                                                                   ERR_PARAM, 
                                                                   model_instance, 
                                                                   error_profile_tag, 
                                                                   this_seed)
                # using best chromosome
                shuffle_orders[error_layer] = best
                eval_acc  = eval_error_injected_with_shuffle(model, 
                                                              error_profiles, 
                                                              ERR_PARAM,
                                                              shuffle_orders,
                                                              test_set)
                results[model_seed][error_profile_tag][ERR_PARAM][error_layer]["ga_optim"] = eval_acc
                np.save(results_filename, results) 
                print("GA ACC: ", eval_acc)
        print()
    print()

fashion-cnn2_1024-1023
Original ACC:  0.9298930921052632
LIM_500-2188
ERR_PARAM:	 1
Layer:  c0
Faulty ACC:  0.8564967105263158
GA ACC:  0.9052905701754387
Layer:  c1
Faulty ACC:  0.846765350877193
GA ACC:  0.8588267543859649
ERR_PARAM:	 -1
Layer:  c0
Faulty ACC:  0.8497121710526315
GA ACC:  0.9004248903508771
Layer:  c1
Faulty ACC:  0.8000959429824562
GA ACC:  0.8292214912280702

LIM_500-4981
ERR_PARAM:	 1
Layer:  c0
Faulty ACC:  0.8262061403508772
GA ACC:  0.8935718201754387
Layer:  c1
Faulty ACC:  0.8436814692982456
GA ACC:  0.8546463815789473
ERR_PARAM:	 -1
Layer:  c0
Faulty ACC:  0.8196957236842106
GA ACC:  0.8911047149122807
Layer:  c1
Faulty ACC:  0.8020833333333334
GA ACC:  0.8161321271929824


fashion-cnn2_1024-2920
Original ACC:  0.9298930921052632
LIM_500-2188
ERR_PARAM:	 1
Layer:  c0
Faulty ACC:  0.8988486842105264
GA ACC:  0.9091282894736841
Layer:  c1
Faulty ACC:  0.810718201754386
GA ACC:  0.8499862938596491
ERR_PARAM:	 -1
Layer:  c0
Faulty ACC:  0.893640350877193
GA ACC:

In [ ]:
results_filename = 'evaluate_ga_genes-c0_c1--LIM_500-ERR_1_-1.npy'
results = np.load(str(results_filename), allow_pickle='TRUE').item()
# both c0-c1 layer at a time
for model_seed in model_seeds:#model_seeds[0:2]:
    # Get model
    model_instance = model_type + "-" + str(model_seed)
    dataset, model_arch, model_config, layer_widths, seed = utils.instancename2metadata(model_instance)
    model_meta_type, model_type, model_instance = utils.metadata2instancenames(dataset, model_arch, layer_widths, seed)
    model_folder = pathlib.Path(PROJ_ROOT_PATH / "models" / model_type)
    model_filename = model_instance + ".h5"
    model_file = pathlib.Path(model_folder/ model_filename)
    # Load model
    model = tf.keras.models.load_model(model_file)
    print(model_instance)
    
    for error_profile_tag in error_profile_tag_list:
        print(error_profile_tag)
        # Load error profile
        error_profile_folder = pathlib.Path(PROJ_ROOT_PATH / "error_profiles")
        error_profile_filename = error_profile_tag + ".npy"
        error_profile_file = pathlib.Path(error_profile_folder/ error_profile_filename)
        error_profile = np.load(error_profile_file)

        for ERR_PARAM in ERR_PARAM_list:
            print("ERR_PARAM:\t", ERR_PARAM)
            new_error_layer = "c0-c1"
            print("Layer: ", new_error_layer)
            
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]={}
            # Reset error profiles and shuffle orders for all layers
            error_profiles = reset_dict_values()
            shuffle_orders = reset_dict_values()
            
            # Inject error in BOTH layers and observe degradation
            for error_layer in error_layer_list:
                error_profiles[error_layer] = error_profile

            eval_acc  = eval_error_injected_with_shuffle(model, 
                                                          error_profiles, 
                                                          ERR_PARAM,
                                                          shuffle_orders,
                                                          test_set)
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]["faulty"] = eval_acc
            np.save(results_filename, results) 
            print("Faulty ACC: ", eval_acc)
            
            # Get chromosomes for both layers
            for error_layer in error_layer_list:
                # Use the best chromosome and observe recovered value
                best, worst = extract_best_worst_chromosomes(dataset_seg_type, # (all, qmean1, qent2)
                                                                   error_layer, 
                                                                   ERR_PARAM, 
                                                                   model_instance, 
                                                                   error_profile_tag, 
                                                                   this_seed)
                # using best chromosome
                shuffle_orders[error_layer] = best
            eval_acc  = eval_error_injected_with_shuffle(model, 
                                                          error_profiles, 
                                                          ERR_PARAM,
                                                          shuffle_orders,
                                                          test_set)
            results[model_seed][error_profile_tag][ERR_PARAM][new_error_layer]["ga_optim"] = eval_acc
            np.save(results_filename, results) 
            print("GA ACC: ", eval_acc)
        print()
    print()

fashion-cnn2_1024-1023
LIM_500-2188
ERR_PARAM:	 1
Layer:  c0-c1
Faulty ACC:  0.7657620614035087
GA ACC:  0.8297697368421053
ERR_PARAM:	 -1
Layer:  c0-c1
